![ine-divider](https://user-images.githubusercontent.com/7065401/92672068-398e8080-f2ee-11ea-82d6-ad53f7feb5c0.png)
<hr>

# PostgreSQL for Python Developers

## User-defined functions on PostgreSQL

In this project, you will enhance PostgreSQL to perform operations not available by default in queries.

You will need access to a PostgreSQL installation where you have superuser permissions. If you do not have such access elsewhere, installing to your personal workstation is a good idea.  Alternately, you might wish to use a Docker container for a self-contained installation.  See `https://hub.docker.com/_/postgres` for details on that option.  Unless you have a specific need to work with an existing installation, choosing a PostgreSQL version of 12 or higher is best.

![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)

## Part 1

**Calculating geometric mean**

In this task, we will continue to use the airline tweets table that has been used in previous projects.

In [1]:
import pandas as pd
import psycopg2
cred = dict(user='ine_student', password='ine-password', database='ine', host='localhost')
conn = psycopg2.connect(**cred)
cur = conn.cursor()

If you don't have the database already created, create it:

In [ ]:
# create PostgreSQL table
sql_create_tweets = '''
CREATE TABLE IF NOT EXISTS Tweets (
    tweet_id DECIMAL(18) PRIMARY KEY,
    airline_sentiment TEXT,
    airline_sentiment_confidence REAL,
    negativereason TEXT,
    negativereason_confidence REAL,
    airline TEXT,
    airline_sentiment_gold TEXT,
    name TEXT,
    negativereason_gold TEXT,
    retweet_count INT,
    text TEXT,
    tweet_coord TEXT,
    tweet_created TIMESTAMP WITH TIME ZONE,
    tweet_location TEXT,
    user_timezone TEXT
    );
'''
cur_dest.execute('DROP TABLE Tweets')
cur_dest.execute(sql_create_tweets)
con_dest.commit()

In [ ]:
con_src = sqlite3.connect('data/Airline-Tweets.sqlite') 
cur_src = con_src.cursor()
cur_src.execute("SELECT * FROM Tweets")

In [ ]:
sql_insert = """
INSERT INTO Tweets 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
for src_row in cur_src:
    row = tuple(data or None for data in src_row)
    cur_dest.execute(sql_insert, row)
    
con_dest.commit()

Sometimes we with to perform queries that describe aggregations of the data.  For example:

In [2]:
sql = """
SELECT airline, avg(airline_sentiment_confidence), avg(negativereason_confidence)
FROM tweets
GROUP BY airline;
"""
cur.execute(sql)
pd.DataFrame(cur.fetchall(), columns=['airline', 'avg_sentiment_conf', 'avg_neg_conf'])

,airline,avg_sentiment_conf,avg_neg_conf
0,Virgin America,0.838760,0.556985
1,Southwest,0.867695,0.601508
2,Delta,0.847048,0.573097
3,American,0.912919,0.665752
4,US Airways,0.919169,0.687172
5,United,0.896252,0.627285


For this task, you would like to make a similar report, but using geometric mean rather than arithmetic mean.  Since PostgreSQL does not include a function for geometric mean in a default installation, you will have to create one.  In the PosgreSQL documentation, there is discussion of aggregations beyond what is provided in the lesson itself. See:

> https://www.postgresql.org/docs/current/xaggr.html

As well, you may wish to think about the Python `statistics` module for an implementation of geometric mean:

> https://docs.python.org/3/library/statistics.html

**Solution**

In [3]:
sql_accum = """
CREATE OR REPLACE FUNCTION gm_accum(logsum float8[], num float8)
RETURNS float8[] LANGUAGE sql
AS $$ SELECT array[logsum[1]+ln(num), logsum[2]+1.0] $$;
"""

sql_final = """
CREATE OR REPLACE FUNCTION gm_final(logsum float8[])
RETURNS float8 LANGUAGE sql
AS $$ SELECT exp(logsum[1]/logsum[2]) $$;
"""

sql_agg = """
CREATE OR REPLACE AGGREGATE gm(float8) (
    sfunc = gm_accum,
    stype = float8[],
    finalfunc = gm_final,
    initcond = '{0.0, 0.0}'
);
"""
cur.execute(sql_accum)
cur.execute(sql_final)
cur.execute(sql_agg)

In [4]:
sql = """
SELECT airline, gm(airline_sentiment_confidence), gm(negativereason_confidence)
FROM tweets
WHERE airline_sentiment_confidence > 0
AND negativereason_confidence > 0
GROUP BY airline;
"""
cur.execute(sql)
pd.DataFrame(cur.fetchall(), columns=['airline', 'gm_sentiment_conf', 'gm_neg_conf'])

,airline,gm_sentiment_conf,gm_neg_conf
0,Virgin America,0.885061,0.675881
1,Southwest,0.905992,0.688806
2,Delta,0.883982,0.664280
3,American,0.934933,0.700478
4,US Airways,0.935062,0.705560
5,United,0.920436,0.668733


![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)

## Part 2

**Create a view into averages**

For this task, we suppose that users frequently wish to see the average confidence about sentiment about airlines.  The would like to be able to query both geometric and arithmetic means as if they are simple columns.  E.g.

```
ine=# SELECT * FROM airline_confidences

 airline        |  gm_conf | gm_neg_conf | avg_conf | avg_neg_conf
----------------+----------+-------------+----------+-------------
 Virgin America | 0.885061 | 0.675881    | 0.901733 | 0.717003
 Southwest      | 0.905992 | 0.688806    | 0.920533 | 0.732866
 Delta          | 0.883982 | 0.66428     | 0.902202 | 0.71052 
 American       | 0.934933 | 0.700478    | 0.945037 | 0.744327
 US Airways     | 0.935062 | 0.70556     | 0.945714 | 0.750028
 United         | 0.920436 | 0.668733    | 0.933383 | 0.714719
```

**Solution**

In [5]:
sql_conf = """
CREATE OR REPLACE VIEW airline_confidences (airline, gm_conf, gm_neg_conf, avg_conf, avg_neg_conf) AS 
SELECT airline, gm(airline_sentiment_confidence), gm(negativereason_confidence),
                avg(airline_sentiment_confidence), avg(negativereason_confidence)
FROM tweets
WHERE airline_sentiment_confidence > 0
AND negativereason_confidence > 0
GROUP BY airline;
"""
cur.execute(sql_conf)

In [6]:
cur.execute("SELECT * FROM airline_confidences;")
pd.DataFrame(cur.fetchall(), columns=[c.name for c in cur.description])

,airline,gm_conf,gm_neg_conf,avg_conf,avg_neg_conf
0,Virgin America,0.885061,0.675881,0.901733,0.717003
1,Southwest,0.905992,0.688806,0.920533,0.732866
2,Delta,0.883982,0.664280,0.902202,0.710520
3,American,0.934933,0.700478,0.945037,0.744327
4,US Airways,0.935062,0.705560,0.945714,0.750028
5,United,0.920436,0.668733,0.933383,0.714719


In [7]:
# Normally would commit, but rollback here
conn.rollback()

![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)